# Introduction

General convex programming frameworks (e.g. CVX) have historically relied on transforming any optimization problem into a canonical cone form and solving them with generic cone solvers. The appeal of this approach is that a large set of convex functions can be written with a small set of atoms (namely, linear functions and cone constraints) allowing them to be solved by robust algorithms (e.g. interior point methods) after a transformation step.

The goal of Epsilon is to address the gap between the flexibility of general convex programming frameworks and the speed and scalability of achievable with specialized algorithms. The key innovations that enable progress toward this goal are 1) considering new problem forms beyond the canonical cone form; which enables 2) a dramatic expansion of the set of atoms that can be used as the "backend" for general convex programming. By increasing the set of atoms and problems forms we widen the set of applicable numerical methods far beyond the classical conic solvers. 

The first problem form we target is inspired by the popularity of operating splitting methods, e.g. the Alternating Direction Method of Multipliers (ADMM); the *sum-of-prox form* which consists of an objective which is the sum of functions with efficient proximal operators. Many functions have efficient proximal operators and indeed the sum-of-prox form can be seen as a generalization of the canonical cone which requires only 3 proximal operators (projection onto the nonnegative, second-order and semidefinite cones).

While proximal and epigraph operators provide efficient atomic operations for non-linear functions, the same principle also applies to the implementation of linear functions. In particular, existing general convex solvers require linear functions to be instantiated explicitly using either sparse or dense matrices which can be highly inefficient. For example, explicitly instantiating the Kronecker product (required to handle multiplicaiton with matrix-valued variables) is hugely expensive when the matrix needs to be inverted while other linear operators can resulting poorly conditioned matrices but have highly efficient exact solutions (in this case, using FFT).

For both linear and non-linear operators, the challenge in expasnding the set of problem forms and atoms is that it requires a more sophisticated approach to the symbolic manipulation of optimization problems by the compiler frontend. Concretely, there are often many ways to represent a given problem which are all mathematically equivalent but have dramatically different computational properties due to the computational complexity of instantiating different atoms on the backend or the number of iterations required for convergence.

The focus of this notebook is *symbolic* representation and manipulation of the optimization problem, i.e. the main function of the Epsilon compiler frontend.   

# Atoms

In order to solve an optimization problem with numerical algorithms, the problem must be written in a particular form which results in instantiating a particular set of *atoms* backed by numerical routines on the backend. 

(Note that our use of the word "atom" is somewhat different than how it has been used in other convex programming frameworks such as CVXPY and Convex.jl where functions that may be implemented in terms of other functions are still called "atoms". However, the original CVX and DCP paper (Grant, 2006) use the word atom in the same way as we intend here: to refer to an atomic function that does not require a different form to be solved by numerical algorithms.)

## Non-linear functions

In Epsilon, we treat non-linear functions as atomic if there exists an efficient implementation for their proximal operator 

$$ \arg \min_x \lambda f(x) + (1/2)\|x - v\|^2 $$
on the backend. We have implemented (or are in the process of implementing) proximal and epigraph operators implemented for several functions, see https://github.com/mwytock/epsilon/blob/master/Roadmap.txt for the full list.

## Linear functions 

A linear function is treated as atomic if there exists an efficient implementation for the basic operations of a linear map
- $y$ = apply($A$, $x$)
- $B$ = transpose($A$) 
- $B$ = inverse($A$) (with hints, e.g. to indicate inverting symmetric positive definite matrix)

where $A, B$ are linear maps and a $x, y$ are vectors.

Clasically general optimization frameworks support only
- Dense matrices
- Sparse matrices

while specialized solvers often typically take advantage of problem-specific structure to achieve significant speed and scalability improvements. In contrast, Epsilon supports (or will support):
- Scaling (scalar matrix) 
- Elementwise scaling (diagonal matrix)
- Kronecker product - useful for matrix-matrix multiplication
- Convolution (coming soon) - can represent the finite difference operator
- Sequence (coming soon) - useful for sparse matrices with $A$ and $B$ are sparse but $AB$ is not 

In addition linear maps support addition and multiplication/composition binary operators, e.g. 
- $A + B$
- $AB$

which are implemented with multiple dispatch and type coercion, so that (for example) scalar + dense = dense but inverse(kronecker) = kronecker and kronecker + scaling = kronecker. 

Finally, Epsilon supports a general block structure for linear maps (implemented using the notion of linear operators between spaces defined as key-value pairs). This is notationally convenient for representing a heteregeneous mix of linear operators as is required when solving optimization problems. For block linear maps, the apply() and transpose() functions have obvious implementations but inverse() requires a blockwise algorithm.

# Compilation examples

Here we demonstrate the symbolic manipulation of optimizaiton problems starting from the input representation which is virtually equivalent to the form typed by the user except for some minor modifications performed by CVXPY before the problem is passed to Epsilon. 

The first compilation step (canonicalization) transforms the problem to be in sum-of-prox form.

The second compilation step (separation) introduces variable copies and equality constraints so that the problem can be solved using the sum-of-prox ADMM algorithm.

In [14]:
from epsilon import text_format
from epsilon import cvxpy_expr
from epsilon.compiler import canonicalize
from epsilon.compiler import combine
from epsilon.problems import lasso
from epsilon.problems import quantile

def print_problem(problem):
    print "CVXPY input:"
    print text_format.format(problem)
    
    problem = canonicalize.transform(problem)
    print "\nAfter canonicalization:"
    print text_format.format(problem)
    
    problem = combine.transform(problem)
    print "\nAfter separation:"
    print text_format.format(problem)

## Lasso

The lasso problem
$$ \mathrm{minimize} \;\; \|Ax - b\|_2^2 + \lambda\|x\|_1 $$
has relatively simple structure owing to the fact that the objective term is unconstrained and composed of two functions with efficient proximal opeators (least squares and $\ell_1$-norm). The compiler simply introduces a variable copy to make this objective separable.

In [24]:
print_problem(cvxpy_expr.convert_problem(lasso.create(m=5, n=10))[0])

CVXPY input:
problem(
add(
  quad_over_lin(
    add(
      multiply(
        A,
        x),
      negate(b)),
    1.00),
  multiply(
    0.33,
    norm_p[1](x))))

After canonicalization:
problem(
add(
  sum(
    power[2](
      add(
        multiply(
          A,
          x),
        negate(b)))),
  multiply(
    0.33,
    norm_p[1](x))))

After separation:
problem(
add(
  sum(
    power[2](
      add(
        multiply(
          A,
          x),
        negate(b)))),
  multiply(
    0.33,
    norm_p[1](y))),
[zero(
  add(
    x,
    negate(y)))])


## Multiple quantile regression

The multiple quantile regression problem involves an asymmetric $\ell_1$ penalty and the differencing operator to ensure that the quantiles are monotonic.

$$ \begin{split} 
\mathrm{minimize} \;\; & \sum_{j=1}^k \ell(X\theta_i; \alpha_i) \\ 
\mathrm{subject\;to} \;\; & X(\theta_{i+1} - \theta_i) \ge 0 \;\; i = 1,\ldots,k-1 \end{split}$$

where $\ell(y; \alpha) = \max\{-\alpha y, (1-\alpha)y\} $ is the quantile loss function.

In [23]:
print_problem(cvxpy_expr.convert_problem(quantile.create(m=40, n=2, k=3))[0])

CVXPY input:
problem(
add(
  0.00,
  sum(
    max_elementwise(
      multiply(
        -0.25,
        add(
          index[0:40, 0:1](X),
          negate(a))),
      multiply(
        0.75,
        add(
          index[0:40, 0:1](X),
          negate(a))))),
  sum(
    max_elementwise(
      multiply(
        -0.50,
        add(
          index[0:40, 1:2](X),
          negate(a))),
      multiply(
        0.50,
        add(
          index[0:40, 1:2](X),
          negate(a))))),
  sum(
    max_elementwise(
      multiply(
        -0.75,
        add(
          index[0:40, 2:3](X),
          negate(a))),
      multiply(
        0.25,
        add(
          index[0:40, 2:3](X),
          negate(a)))))),
[zero(
  add(
    X,
    negate(
      multiply(
        B,
        Y)))),
non_negative(
  add(
    add(
      index[0:40, 0:2](X),
      negate(
        index[0:40, 1:3](X))),
    negate(0.00)))])

After canonicalization:
problem(
add(
  0.00,
  scaled_zone[alpha=0.75, beta=0.25, C=0.00,

# Lisp, syntax and languages

The optimization problems written above simply consist of serialized expression trees which is nearly the form that users are accustomed to typing them (e.g. in CVXPY) with the exception that binary operators for * and + have been replaced with the more verbose add() and multiply(). This is happens "automatically" using operating overloading in Python.

To emphasize the symbolic nature of the compiler frontend, we can just as well format the expression trees as lists resulting in a lisp-like syntax (replacing parenthesis with brackets).

In [18]:
from epsilon import list_format
problem = cvxpy_expr.convert_problem(lasso.create(m=5, n=10))[0]
list_format.format(problem)

['problem',
 ['add',
  [],
  [['quad_over_lin',
    [],
    [['add',
      [],
      [['multiply',
        [],
        [['constant', ['data_location', u'/mem/data/4539540410267417205'], []],
         ['variable', ['variable_id', u'cvxpy:7'], []]]],
       ['negate',
        [],
        [['constant',
          ['data_location', u'/mem/data/6448201545586576587'],
          []]]]]],
     ['constant', ['scalar', 1], []]]],
   ['multiply',
    [],
    [['constant', ['scalar', 0.32523071495341488], []],
     ['norm_p', ['p', 1], [['variable', ['variable_id', u'cvxpy:7'], []]]]]]]],
 []]

We could actually write the entire compiler frontend itself in Lisp although writing it in Python (as we have done so far) may be just as good, see e.g. http://norvig.com/lispy.html. 

Regardless, the [Expression abstract syntax tree](https://github.com/mwytock/epsilon/blob/master/proto/epsilon/expression.proto) is language agnostic which emphasizes the fact that the compiled language and the interface language are decoupled, unlike existing convex programming frameworks. 

In fact, given a new grammar for convex optimization problems (e.g. in EBNF) we could easily write a parser that generates abstract syntax trees in our existing form. This could also be coupled with an interpreter and a REPL loop although there doesnt seem to be a major reason to do this given the existance of CVXPY and the numerical environment provided by Python.

# Killer language feature?

One major limitation of CVXPY is that it unrolls all of the loops which makes it impractical to use loops for any size requiring the use to instead express everything in terms of matrices. This is despite the fact that Python with list comprehensions, etc. makes it very easy to write loops in a concise and idiomatic manner. New users to CVXPY frequently get bit by this when they hit performance problems and its a frequent topic of discussion on the CVXPY mailing list. 

In general, a programming language that does not support loops is rather weak. Imagine if C had this limitation! 

In order to support loops in the right way, you would most likely need a new programming language (or you _may_ be able to build it onto of python generators, but this seems tricky). The key limitation is that langauge-level constructs in Python are not available to application code. Essentially, Python does not support Lisp-like macros so the only option is to invent a new construct for a for-loop (e.g. "for(0, 1, 10, ...)") which would be so ugly as to completely defeat the purpose.

It turns out Julia does support macros but Convex.jl does not handle loops by incorporating them into the problem description problably because 1) there may be minor benefit w/ existing solvers and 2) Julia does not have the same list comprehension idioms as Python (I don't know if this is true or not).

In any event, a dedicated language could easily support this by supporting a loop construct in the abstract syntax tree. The syntax for loops would have to be defined but likely this could borrow heavily from the Python version.  